In [1]:
import random
import warnings
import keras as k
import numpy as np
import pandas as pd
import seaborn as sns
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from models.linear_regression.linear_regression import LinearRegression as LR
import plotly
import plotly.graph_objs as go

import config as c
from sklearn.model_selection import train_test_split
from pipeline.Layer import Layer
from utils.cuda import turn_off_gpu
from models.svm.svm import SVM
from utils.metrics import roc_auc_score_at_K
from utils.preprocess import preprocess 
from models.keras_dense_classifier.keras_dense_classifier import KerasDenseClassifier as KDC
from visualization.utils import plot_correlation_matrix, plot_scatterplot_matrix

turn_off_gpu()
init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')
plt.rcParams ['figure.figsize'] = (13,8)
sns.set()
%config InlineBackend.figure_format = 'retina'

In [248]:
import pandas as pd


def preprocess(df):
    cols = [el for el in list(df.select_dtypes(include=['object']).columns) if el != 'card_id']
    df = pd.get_dummies(df, columns=cols)
    df.drop([c for c in df.columns if len(df[c].value_counts().values) == 2 and df[c].value_counts().values[1] < 50], axis=1)

    df = df.fillna(0)

    return df

In [2]:
df = pd.read_csv('train.csv', sep=',')
pd.set_option('display.max_columns', 500)
df['addr_region_fact_encoding2'] = (df['addr_region_fact_encoding2']*11).round(0).astype(int)
df['addr_region_fact_encoding1'] = (df['addr_region_fact_encoding1']*83).round(0).astype(int)
df['addr_region_reg_encoding1'] = (df['addr_region_reg_encoding1']*83).round(0).astype(int)
df['addr_region_reg_encoding2'] = (df['addr_region_reg_encoding2']*11).round(0).astype(int)
df['app_addr_region_reg_encoding2'] = (df['app_addr_region_reg_encoding2']*11).round(0).astype(int)
df['app_addr_region_reg_encoding1'] = (df['app_addr_region_reg_encoding1']*83).round(0).astype(int)
df['app_addr_region_fact_encoding1'] = (df['app_addr_region_fact_encoding1']*83).round(0).astype(int)
df['app_addr_region_fact_encoding2'] = (df['app_addr_region_fact_encoding2']*11).round(0).astype(int)
df['app_addr_region_sale_encoding1'] = (df['app_addr_region_sale_encoding1']*39).round(0).astype(int)
df['app_addr_region_sale_encoding2'] = (df['app_addr_region_sale_encoding2']*7).round(0).astype(int)

df = preprocess(df)
df.head()

,card_id,target,addr_region_reg,addr_region_fact,sas_limit_after_003_amt,sas_limit_last_amt,clnt_income_month_avg_net_amt,clnt_expense_month_avg_amt,clnt_experience_cur_mnth,clnt_experience_cur_year,clnt_experience_total_mnth,app_addr_region_reg,app_addr_region_fact,app_addr_region_sale,clnt_birth_year,addr_region_fact_encoding1,addr_region_fact_encoding2,addr_region_reg_encoding1,addr_region_reg_encoding2,app_addr_region_reg_encoding1,app_addr_region_reg_encoding2,app_addr_region_fact_encoding1,app_addr_region_fact_encoding2,app_addr_region_sale_encoding1,app_addr_region_sale_encoding2,loans_main_borrower,loans_active,last_loan_date,first_loan_date,max_overdue_status,ttl_officials,ttl_legals,ttl_bankruptcies,inquiry_recent_period,inquiry_3_month,inquiry_6_month,inquiry_9_month,inquiry_12_month,ttl_inquiries,ttl_auto_loan,ttl_mortgage,ttl_credit_card,ttl_consumer,worst_status_ever,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,fl_coborrower,fl_active_coborrower,pay_load,inquiry_1_week,inquiry_1_month,feature_10,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,makro_region,fo,region,feature_30,delivery_type_cat_1,delivery_type_cat_2,delivery_type_cat_3,channel_name_cat_0,channel_name_cat_1,channel_name_cat_2,channel_name_cat_3,channel_name_cat_4,channel_name_cat_5,channel_name_cat_6,channel_name_2_cat_0,channel_name_2_cat_1,channel_name_2_cat_2,channel_name_2_cat_3,channel_name_2_cat_4,channel_name_2_cat_5,channel_name_modified_2018_cat_0,channel_name_modified_2018_cat_1,channel_name_modified_2018_cat_2,channel_name_modified_2018_cat_3,channel_name_modified_2018_cat_4,clnt_education_name_cat_0,clnt_education_name_cat_1,clnt_education_name_cat_2,clnt_education_name_cat_3,clnt_education_name_cat_4,clnt_education_name_cat_5,clnt_education_name_cat_6,clnt_marital_status_name_cat_0,clnt_marital_status_name_cat_1,clnt_marital_status_name_cat_2,clnt_marital_status_name_cat_3,clnt_marital_status_name_cat_4,clnt_employment_type_name_cat_0,clnt_employment_type_name_cat_1,clnt_employment_type_name_cat_2,clnt_employment_type_name_cat_3,clnt_employment_type_name_cat_4,clnt_speciality_sphere_name_cat_0,clnt_speciality_sphere_name_cat_1,clnt_speciality_sphere_name_cat_10,clnt_speciality_sphere_name_cat_11,clnt_speciality_sphere_name_cat_12,clnt_speciality_sphere_name_cat_13,clnt_speciality_sphere_name_cat_14,clnt_speciality_sphere_name_cat_15,clnt_speciality_sphere_name_cat_16,clnt_speciality_sphere_name_cat_17,clnt_speciality_sphere_name_cat_18,clnt_speciality_sphere_name_cat_19,clnt_speciality_sphere_name_cat_2,clnt_speciality_sphere_name_cat_20,clnt_speciality_sphere_name_cat_21,clnt_speciality_sphere_name_cat_22,clnt_speciality_sphere_name_cat_23,clnt_speciality_sphere_name_cat_24,clnt_speciality_sphere_name_cat_25,clnt_speciality_sphere_name_cat_26,clnt_speciality_sphere_name_cat_27,clnt_speciality_sphere_name_cat_28,clnt_speciality_sphere_name_cat_3,clnt_speciality_sphere_name_cat_4,clnt_speciality_sphere_name_cat_5,clnt_speciality_sphere_name_cat_6,clnt_speciality_sphere_name_cat_7,clnt_speciality_sphere_name_cat_8,clnt_speciality_sphere_name_cat_9,clnt_sex_name_cat_0,clnt_sex_name_cat_1,prt_name_cat_0,prt_name_cat_1,prt_name_cat_2,prt_name_cat_3,prt_name_cat_4,prt_name_cat_5,prt_name_cat_6,prt_name_cat_7,prt_name_cat_8,prt_name_cat_9,feature_0_cat_0,feature_0_cat_1,feature_0_cat_10,feature_0_cat_12,feature_0_cat_13,feature_0_cat_14,feature_0_cat_15,feature_0_cat_16,feature_0_cat_3,feature_0_cat_6,feature_0_cat_7,feature_0_cat_8,feature_0_cat_9
0,cid_10620,1,107,107,1,1,3,0,0.0,0.0,0.0,107,107,45,46,560000,78750,560000,78750,560000,78750,560000,78750,260000,46000,0.88,0.78,193.0,2851.0,2,0.0,0.0,0.0,0.0,7.0,10.0,12.0,19.0,87,0,0,1,1,3,3,10,4,0,0,5,1,0,0,0,0,0.82,1.0,3.0,0.57,0.64,0.7,0.0,0.0,0.00,0.00,0.57,0.88,0.0,0.00,0.77,0.49,0.53,0.91,0.99,0.

In [4]:
df = pd.read_csv('train.csv', sep=',')
pd.set_option('display.max_columns', 500)
df['addr_region_fact_encoding2'] = (df['addr_region_fact_encoding2']*11).round(0).astype(int)
df['addr_region_fact_encoding1'] = (df['addr_region_fact_encoding1']*83).round(0).astype(int)
df['addr_region_reg_encoding1'] = (df['addr_region_reg_encoding1']*83).round(0).astype(int)
df['addr_region_reg_encoding2'] = (df['addr_region_reg_encoding2']*11).round(0).astype(int)
df['app_addr_region_reg_encoding2'] = (df['app_addr_region_reg_encoding2']*11).round(0).astype(int)
df['app_addr_region_reg_encoding1'] = (df['app_addr_region_reg_encoding1']*83).round(0).astype(int)
df['app_addr_region_fact_encoding1'] = (df['app_addr_region_fact_encoding1']*83).round(0).astype(int)
df['app_addr_region_fact_encoding2'] = (df['app_addr_region_fact_encoding2']*11).round(0).astype(int)
df['app_addr_region_sale_encoding1'] = (df['app_addr_region_sale_encoding1']*39).round(0).astype(int)
df['app_addr_region_sale_encoding2'] = (df['app_addr_region_sale_encoding2']*7).round(0).astype(int)

df = preprocess(df[:100], False)
df.head()

,card_id,target,addr_region_reg,addr_region_fact,sas_limit_after_003_amt,sas_limit_last_amt,clnt_income_month_avg_net_amt,clnt_expense_month_avg_amt,clnt_experience_cur_mnth,clnt_experience_cur_year,clnt_experience_total_mnth,app_addr_region_reg,app_addr_region_fact,app_addr_region_sale,clnt_birth_year,addr_region_fact_encoding1,addr_region_fact_encoding2,addr_region_reg_encoding1,addr_region_reg_encoding2,app_addr_region_reg_encoding1,app_addr_region_reg_encoding2,app_addr_region_fact_encoding1,app_addr_region_fact_encoding2,app_addr_region_sale_encoding1,app_addr_region_sale_encoding2,loans_main_borrower,loans_active,last_loan_date,first_loan_date,max_overdue_status,ttl_officials,ttl_legals,ttl_bankruptcies,inquiry_recent_period,inquiry_3_month,inquiry_6_month,inquiry_9_month,inquiry_12_month,ttl_inquiries,ttl_auto_loan,ttl_mortgage,ttl_credit_card,ttl_consumer,worst_status_ever,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,fl_coborrower,fl_active_coborrower,pay_load,inquiry_1_week,inquiry_1_month,feature_10,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,makro_region,fo,region,feature_30,delivery_type_cat_1,delivery_type_cat_3,delivery_type_cat_2,channel_name_cat_0,channel_name_cat_2,channel_name_cat_4,channel_name_cat_6,channel_name_cat_5,channel_name_cat_3,channel_name_cat_1,channel_name_2_cat_3,channel_name_2_cat_5,channel_name_2_cat_0,channel_name_2_cat_4,channel_name_2_cat_2,channel_name_2_cat_1,channel_name_modified_2018_cat_0,channel_name_modified_2018_cat_2,channel_name_modified_2018_cat_4,channel_name_modified_2018_cat_3,channel_name_modified_2018_cat_1,clnt_education_name_cat_1,clnt_education_name_cat_4,clnt_education_name_cat_5,clnt_education_name_cat_0,clnt_education_name_cat_3,clnt_education_name_cat_2,clnt_education_name_cat_6,clnt_marital_status_name_cat_4,clnt_marital_status_name_cat_2,clnt_marital_status_name_cat_3,clnt_marital_status_name_cat_1,clnt_marital_status_name_cat_0,clnt_employment_type_name_cat_0,clnt_employment_type_name_cat_3,clnt_employment_type_name_cat_2,clnt_employment_type_name_cat_1,clnt_employment_type_name_cat_4,clnt_speciality_sphere_name_cat_0,clnt_speciality_sphere_name_cat_26,clnt_speciality_sphere_name_cat_15,clnt_speciality_sphere_name_cat_14,clnt_speciality_sphere_name_cat_24,clnt_speciality_sphere_name_cat_6,clnt_speciality_sphere_name_cat_5,clnt_speciality_sphere_name_cat_21,clnt_speciality_sphere_name_cat_10,clnt_speciality_sphere_name_cat_17,clnt_speciality_sphere_name_cat_23,clnt_speciality_sphere_name_cat_25,clnt_speciality_sphere_name_cat_16,clnt_speciality_sphere_name_cat_12,clnt_speciality_sphere_name_cat_20,clnt_speciality_sphere_name_cat_8,clnt_speciality_sphere_name_cat_1,clnt_speciality_sphere_name_cat_7,clnt_speciality_sphere_name_cat_28,clnt_speciality_sphere_name_cat_18,clnt_speciality_sphere_name_cat_2,clnt_speciality_sphere_name_cat_13,clnt_speciality_sphere_name_cat_9,clnt_speciality_sphere_name_cat_19,clnt_speciality_sphere_name_cat_11,clnt_speciality_sphere_name_cat_4,clnt_speciality_sphere_name_cat_27,clnt_speciality_sphere_name_cat_22,clnt_speciality_sphere_name_cat_3,clnt_sex_name_cat_0,clnt_sex_name_cat_1,prt_name_cat_9,prt_name_cat_8,prt_name_cat_4,prt_name_cat_5,prt_name_cat_2,prt_name_cat_0,prt_name_cat_3,prt_name_cat_7,prt_name_cat_1,prt_name_cat_6,feature_0_cat_7,feature_0_cat_9,feature_0_cat_10,feature_0_cat_13,feature_0_cat_15,feature_0_cat_16,feature_0_cat_8,feature_0_cat_12,feature_0_cat_14,feature_0_cat_6,feature_0_cat_3,feature_0_cat_1,feature_0_cat_0
0,cid_10620,1,107,107,1,1,3,0,0.0,0.0,0.0,107,107,45,46,560000,78750,560000,78750,560000,78750,560000,78750,260000,46000,0.88,0.78,193.0,2851.0,2,0.0,0.0,0.0,0.0,7.0,10.0,12.0,19.0,87,0,0,1,1,3,3,10,4,0,0,5,1,0,0,0,0,0.82,1.0,3.0,0.57,0.64,0.7,0.0,0.0,0.00,0.00,0.57,0.88,0.0,0.00,0.77,0.49,0.53,0.91,0.99,0.

In [ ]:
#df=pd.get_dummies(df, columns=['addr_region_fact', 'addr_region_reg'])

In [5]:
df

,card_id,target,addr_region_reg,addr_region_fact,sas_limit_after_003_amt,sas_limit_last_amt,clnt_income_month_avg_net_amt,clnt_expense_month_avg_amt,clnt_experience_cur_mnth,clnt_experience_cur_year,clnt_experience_total_mnth,app_addr_region_reg,app_addr_region_fact,app_addr_region_sale,clnt_birth_year,addr_region_fact_encoding1,addr_region_fact_encoding2,addr_region_reg_encoding1,addr_region_reg_encoding2,app_addr_region_reg_encoding1,app_addr_region_reg_encoding2,app_addr_region_fact_encoding1,app_addr_region_fact_encoding2,app_addr_region_sale_encoding1,app_addr_region_sale_encoding2,loans_main_borrower,loans_active,last_loan_date,first_loan_date,max_overdue_status,ttl_officials,ttl_legals,ttl_bankruptcies,inquiry_recent_period,inquiry_3_month,inquiry_6_month,inquiry_9_month,inquiry_12_month,ttl_inquiries,ttl_auto_loan,ttl_mortgage,ttl_credit_card,ttl_consumer,worst_status_ever,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,fl_coborrower,fl_active_coborrower,pay_load,inquiry_1_week,inquiry_1_month,feature_10,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,makro_region,fo,region,feature_30,delivery_type_cat_1,delivery_type_cat_3,delivery_type_cat_2,channel_name_cat_0,channel_name_cat_2,channel_name_cat_4,channel_name_cat_6,channel_name_cat_5,channel_name_cat_3,channel_name_cat_1,channel_name_2_cat_3,channel_name_2_cat_5,channel_name_2_cat_0,channel_name_2_cat_4,channel_name_2_cat_2,channel_name_2_cat_1,channel_name_modified_2018_cat_0,channel_name_modified_2018_cat_2,channel_name_modified_2018_cat_4,channel_name_modified_2018_cat_3,channel_name_modified_2018_cat_1,clnt_education_name_cat_1,clnt_education_name_cat_4,clnt_education_name_cat_5,clnt_education_name_cat_0,clnt_education_name_cat_3,clnt_education_name_cat_2,clnt_education_name_cat_6,clnt_marital_status_name_cat_4,clnt_marital_status_name_cat_2,clnt_marital_status_name_cat_3,clnt_marital_status_name_cat_1,clnt_marital_status_name_cat_0,clnt_employment_type_name_cat_0,clnt_employment_type_name_cat_3,clnt_employment_type_name_cat_2,clnt_employment_type_name_cat_1,clnt_employment_type_name_cat_4,clnt_speciality_sphere_name_cat_0,clnt_speciality_sphere_name_cat_26,clnt_speciality_sphere_name_cat_15,clnt_speciality_sphere_name_cat_14,clnt_speciality_sphere_name_cat_24,clnt_speciality_sphere_name_cat_6,clnt_speciality_sphere_name_cat_5,clnt_speciality_sphere_name_cat_21,clnt_speciality_sphere_name_cat_10,clnt_speciality_sphere_name_cat_17,clnt_speciality_sphere_name_cat_23,clnt_speciality_sphere_name_cat_25,clnt_speciality_sphere_name_cat_16,clnt_speciality_sphere_name_cat_12,clnt_speciality_sphere_name_cat_20,clnt_speciality_sphere_name_cat_8,clnt_speciality_sphere_name_cat_1,clnt_speciality_sphere_name_cat_7,clnt_speciality_sphere_name_cat_28,clnt_speciality_sphere_name_cat_18,clnt_speciality_sphere_name_cat_2,clnt_speciality_sphere_name_cat_13,clnt_speciality_sphere_name_cat_9,clnt_speciality_sphere_name_cat_19,clnt_speciality_sphere_name_cat_11,clnt_speciality_sphere_name_cat_4,clnt_speciality_sphere_name_cat_27,clnt_speciality_sphere_name_cat_22,clnt_speciality_sphere_name_cat_3,clnt_sex_name_cat_0,clnt_sex_name_cat_1,prt_name_cat_9,prt_name_cat_8,prt_name_cat_4,prt_name_cat_5,prt_name_cat_2,prt_name_cat_0,prt_name_cat_3,prt_name_cat_7,prt_name_cat_1,prt_name_cat_6,feature_0_cat_7,feature_0_cat_9,feature_0_cat_10,feature_0_cat_13,feature_0_cat_15,feature_0_cat_16,feature_0_cat_8,feature_0_cat_12,feature_0_cat_14,feature_0_cat_6,feature_0_cat_3,feature_0_cat_1,feature_0_cat_0
0,cid_10620,1,107,107,1,1,3,0,0.0,0.0,0.0,107,107,45,46,560000,78750,560000,78750,560000,78750,560000,78750,260000,46000,0.88,0.78,193.0,2851.0,2,0.0,0.0,0.0,0.0,7.0,10.0,12.0,19.0,87,0,0,1,1,3,3,10,4,0,0,5,1,0,0,0,0,0.82,1.0,3.0,0.57,0.64,0.7,0.0,0.0,0.00,0.00,0.57,0.88,0.0,0.00,0.77,0.49,0.53,0.91,0.99,0.

In [240]:
X = df[df.columns[2:]].to_numpy()
y = df['target'].to_numpy()

In [241]:
lr = LR()
lr.create_model(LR.default_model_constructor_parameters)
lr.fit_model(X, y, test_size=0.2)
lr.save_model()
roc_auc_score_at_K(lr.predict(X).reshape(-1), y, rate=1.0)

0.7169259074773042

In [242]:
lr = LR()
lr.create_model(LR.default_model_constructor_parameters)
lr.fit_model(X, y, test_size=0.2)
roc_auc_score_at_K(lr.predict(X).reshape(-1), y, rate=1.0)

0.7169259074773042

In [243]:
coef = lr.model.coef_

In [244]:
df.columns[-133]

'ttl_mortgage'

In [245]:
np.argmax(coef)

159

In [246]:
71-len(coef)

-100

In [10]:
lr.save_model()

In [11]:
lr.load_model()

In [12]:
roc_auc_score_at_K(lr.predict(X).reshape(-1), y, rate=1.0)

0.7172896305981364